In [1]:
using Statistics
using Flux
using Flux: onehotbatch, onecold, crossentropy, throttle
using Base.Iterators: repeated
using CuArrays
CuArrays.allowscalar(true)

In [3]:
imgs = Flux.Data.MNIST.images()
labels = Flux.Data.MNIST.labels();
print(typeof(imgs),size(imgs),'\n')
print(typeof(labels),size(labels), '\n')

Array{Array{ColorTypes.Gray{FixedPointNumbers.Normed{UInt8,8}},2},1}(60000,)
Array{Int64,1}(60000,)


In [4]:
rand_int = rand(1:60000) # picking a random number in interval 1-60000
print("Image label: ", labels[rand_int])
imgs[rand_int]

Image label: 5

In [5]:
## Preprocessing
X = hcat(float.(reshape.(imgs, :))...) |> gpu #stack all the images, then convert to CuArray
Y = onehotbatch(labels, 0:9) |> gpu # just a common way to encode categorical variables, then convert to CuArray

10×60000 Flux.OneHotMatrix{CuArray{Flux.OneHotVector,1}}:
 0  1  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  1  0  0  1  0  1  0  0  0  0     0  0  0  0  0  0  1  0  0  0  0  0
 0  0  0  0  0  1  0  0  0  0  0  0  0     0  0  0  1  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  1  0  0  1  0  1     0  0  0  0  0  0  0  0  1  0  0  0
 0  0  1  0  0  0  0  0  0  1  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  1  0  …  0  0  0  0  0  1  0  0  0  1  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     1  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  1  0  0  0  0  0  1  0  0  0  1
 0  0  0  0  1  0  0  0  0  0  0  0  0     0  0  1  0  1  0  0  0  0  0  0  0

┌ Warning: Performing scalar operations on GPU arrays: This is very slow, consider disallowing these operations with `allowscalar(false)`
└ @ GPUArrays /home/dreuter/.julia/packages/GPUArrays/tIMl5/src/indexing.jl:16


In [6]:
print(typeof(X)," with shape ",size(X),'\n')
print(typeof(Y)," with shape ",size(Y),'\n')

CuArray{Float32,2} with shape (784, 60000)
Flux.OneHotMatrix{CuArray{Flux.OneHotVector,1}} with shape (10, 60000)


In [8]:
# Our model, just like before, chaining dense layers
# Go from 28^2 dimensional space (images are 28x28) to 10 dimensional space (labels are 0-9)

m = Chain(
  Dense(28^2, 32, relu),
  Dense(32, 10),
  softmax) |> gpu

# softmax just converts output to probability distribution

Chain(Dense(784, 32, relu), Dense(32, 10), softmax)

In [12]:
loss(x, y) = crossentropy(m(x), y) 
opt = ADAM(); # stochastic gradient descent
accuracy(x, y) = mean(onecold(m(x)) .== onecold(y)) # cute way to find average of correct guesses
dataset = repeated((X,Y),200) # repeat the data set 200 times, as opposed to @epochs 200 ...
print(typeof(dataset))

evalcb = () -> @show(loss(X, Y)); # callback to show loss

Base.Iterators.Take{Base.Iterators.Repeated{Tuple{CuArray{Float32,2},Flux.OneHotMatrix{CuArray{Flux.OneHotVector,1}}}}}

In [20]:
Flux.train!(loss, params(m), dataset, opt, cb = throttle(evalcb, 1)); #~5 minutes to train on CPU

loss(X, Y) = 0.085011765f0 (tracked)
loss(X, Y) = 0.07282233f0 (tracked)
loss(X, Y) = 0.062135816f0 (tracked)
loss(X, Y) = 0.054115478f0 (tracked)
loss(X, Y) = 0.04672469f0 (tracked)


In [135]:
rand_int = rand(1:10000) # picking a random number in interval 1-10000
#print("Image label: ", labels[:test][rand_int])
Flux.Data.MNIST.images(:test)[rand_int]

In [13]:
# Same preprocessing
test_X = hcat(float.(reshape.(Flux.Data.MNIST.images(:test), :))...) |> gpu
test_Y = onehotbatch(Flux.Data.MNIST.labels(:test), 0:9) |> gpu;

m(test_X[:,5287]) # Note the 7th index ( corresponding to the digit 6 ) is nearly 1

Tracked 10-element CuArray{Float32,1}:
 3.502388f-7  
 7.5958957f-7 
 2.0465517f-5 
 1.884268f-7  
 1.5137834f-5 
 2.2860945f-6 
 0.99993134f0 
 2.5564226f-10
 2.9246052f-5 
 2.45021f-7   

In [14]:
#decode
onecold(m(test_X[:,5287])) - 1 #minus 1 since we start from 0, but indexing in Julia starts at 1

6

In [21]:
# Training set accuracy
accuracy(X, Y)

0.9887833333333333

In [22]:
# Test set accuracy
accuracy(test_X, test_Y)

0.9669